# 如何使用千帆 Python SDK 搭配预置大模型服务进行批量推理

在 0.2.8 版本中，千帆 Python SDK 增加了对批量推理的支持，使用该功能需要视情况开通千帆的模型服务，确保您的账号可以调用您想进行批量推理的服务。

# 准备工作

在开始之前，请确保你的千帆 Python SDK 已经升级到了 0.2.8 及以上版本。
nest_asyncio 是一个异步库，用于支持 Python SDK 的异步推理功能。

In [ ]:
# ! pip install -U "qianfan>=0.2.8"
# ! pip install nest_asyncio

并且在环境变量中设置好 Access Key 与 Secret Key

In [1]:
import logging
import os

from qianfan.utils import enable_log

os.environ['QIANFAN_ACCESS_KEY'] = 'your_access_key'
os.environ['QIANFAN_SECRET_KEY'] = 'your_secret_key'

os.environ["QIANFAN_QPS_LIMIT"] = "1"
os.environ['QIANFAN_LLM_API_RETRY_COUNT'] = "3"

# 选择打印出来的日志等级，目前打印出 info 级别
enable_log(logging.INFO)

# 正文

为了开始批量推理，我们首先需要获取到用于做批量推理输入的数据集文件，并且指定用做推理输入的列名

In [2]:
from qianfan.dataset import Dataset

dataset_file_path = "data_file/qa_pair.csv"
dataset_input_column_list = ["prompt"]

# 预期输出列列名，当数据集为对话类数据集时必填，为非对话数据集时选填。
# 对应列的数据会在推理结果中出现
reference_column = "response"

ds = Dataset.load(data_file=dataset_file_path, input_columns=dataset_input_column_list, reference_column=reference_column)

# 预览数据格式
print(ds.list(0))

[INFO] [02-23 12:41:39] dataset.py:489 [t:8599036544]: no data source was provided, construct
[INFO] [02-23 12:41:39] dataset.py:358 [t:8599036544]: construct a file data source from path: data_file/qa_pair.csv, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [02-23 12:41:39] file.py:165 [t:8599036544]: use format type FormatType.Csv
[INFO] [02-23 12:41:39] dataset.py:934 [t:8599036544]: list local dataset data by 0


{'prompt': '地球的自转周期是多久？', 'response': '大约24小时'}


在导入之后，用户可以根据自己的需求，传入不同的参数来使用不同的方式进行推理

In [7]:
#-# cell_skip
# 用户可以设置 service_model 为自己想要的模型名，来直接对数据进行批量推理，以 EB 4 为例
result = ds.test_using_llm(service_model="ERNIE-Bot-4")

# 用户还可以设置 service_endpoint 来使用预置或自己的服务。
result = ds.test_using_llm(service_endpoint="completions_pro")

[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13195636736]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13229215744]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13363531776]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13279584256]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13246005248]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13329952768]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13346742272]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13296373760]: requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 09:55:41] openapi_requestor.py:244 [t:13262794752]

如果用户有异步请求的需求，还可以使用 `atest_using_llm` 来进行异步批量推理

In [3]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

result = asyncio.run(ds.atest_using_llm(service_endpoint="completions_pro"))

[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] oauth.py:229 [t:8599036544]: trying to refresh access_token for ak `rRlk1M***`
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.py:275 [t:8599036544]: async requesting llm api endpoint: /chat/completions_pro
[INFO] [02-23 12:41:42] openapi_requestor.p

KeyboardInterrupt: 

拿到的 `result` 对象也是一个 `Dataset` 对象，可以继续使用千帆 Python SDK 进行后续处理，或者直接保存到本地。

In [4]:
print(result.list(0))
      
dataset_save_file_path = "output_file.csv"

result.save(data_file=dataset_save_file_path)

[INFO] [12-28 13:05:47] dataset.py:981 [t:8120441664]: list local dataset data by 0
[INFO] [12-28 13:05:47] dataset.py:525 [t:8120441664]: no destination data source was provided, construct
[INFO] [12-28 13:05:47] dataset.py:337 [t:8120441664]: construct a file data source from path: output_file.csv, with args: {'is_download_to_local': False}
[INFO] [12-28 13:05:47] data_source.py:280 [t:8120441664]: use format type FormatType.Csv
[INFO] [12-28 13:05:47] dataset.py:233 [t:8120441664]: export as format: FormatType.Csv


{'prompt': '地球的自转周期是多久？', 'input_prompt': '地球的自转周期是多久？', 'llm_output': '地球的自转周期是**24小时**，即一天一夜。这是地球绕自身轴线旋转一周所需的时间。', 'expected_output': '大约24小时'}


True

## 对模型进行批量推理

除了对 `Service` 进行批量推理，我们也可以对 `Model` 进行批量推理

在对 `Model` 进行批量推理时，请先确认用到的数据集已经在千帆平台上发布

In [4]:
cloud_dataset_id = "dataset_id"

qianfan_ds = Dataset.load(qianfan_dataset_id=cloud_dataset_id, is_download_to_local=False)

result = qianfan_ds.test_using_llm(model_version_id="amv-qb8ijukaish3")
print(result[0])

[INFO] [01-05 16:31:19] dataset.py:466 [t:8120441664]: no data source was provided, construct
[INFO] [01-05 16:31:19] dataset.py:350 [t:8120441664]: construct a qianfan data source from existed id: 44478, with args: {'is_download_to_local': False}
[INFO] [01-05 16:31:20] dataset.py:150 [t:8120441664]: a cloud dataset has been created
[INFO] [01-05 16:31:20] dataset_utils.py:353 [t:8120441664]: start to create evaluation task in model
[INFO] [01-05 16:31:24] dataset_utils.py:315 [t:8120441664]: start to polling status of evaluation task 2817
[INFO] [01-05 16:31:24] dataset_utils.py:322 [t:8120441664]: current eval_state: Pending
[INFO] [01-05 16:31:54] dataset_utils.py:322 [t:8120441664]: current eval_state: Doing
[INFO] [01-05 16:32:25] dataset_utils.py:322 [t:8120441664]: current eval_state: Doing
[INFO] [01-05 16:32:55] dataset_utils.py:322 [t:8120441664]: current eval_state: Doing
[INFO] [01-05 16:33:25] dataset_utils.py:322 [t:8120441664]: current eval_state: Doing
[INFO] [01-05 16

{'prompt': '人类的基本单位是什么？', 'input_prompt': '人类的基本单位是什么？', 'llm_output': '人类社会学界、群落、组织、团队、组织、机构、公司、政府、公司、家庭、家庭、军队、学校和家庭、村落、门派、门派、门派、门派、门派是人类的集合。\n\n\n\n\n人类的基本结构层次单位是集合名词，人类的基本单位是集合名词，人类的基本单位是集合名词，人类的基本单位是集合名词，人类的基本单位是个人。\n\n\n人类的基本单位是基本单位是个人。', 'expected_output': '人类'}


# 进阶能力

在调用 `test_using_llm` 时，用户还可以传入一些额外参数来支持额外的功能，比如设置 Prompt 模板，设置人设字段，或者传入大模型调用时的超参数

当对服务进行非对话类推理时，用户可以传入 `prompt_template` 参数来传递一个 Prompt 模板。`prompt_template` 是一个千帆 Python SDK 的 `Prompt` 对象，用户可以通过设置 `Prompt` 对象的 `template` 成员来自定义被用于推理的模板，模板渲染出来的内容将会被作为最终输入提交给大模型。以示例数据集为例，我们可以这么指定一个模板：

In [ ]:
from qianfan.common import Prompt

prompt = Prompt(template="请你就以下问题进行回答: {prompt}")

# 传递给函数
result = ds.test_using_llm(service_model="ERNIE-Bot-4", prompt_template=prompt)

除此之外，用户还可以传入 `system_prompt` 参数来指定对话中大模型需要遵守的人设

In [ ]:
result = ds.test_using_llm(service_model="ERNIE-Bot-4", system_prompt="人设 prompt")

用户在进行批量推理时，还可以直接在 test_using_llm 中传入模型支持的超参数，例如我们可以这么设置模型的 `temperature` :

In [ ]:
result = ds.test_using_llm(service_model="ERNIE-Bot-4", system_prompt="人设 prompt", temperature=0.1)